# Sentiment Adjusted Stock Prediction
By:

Date: /2025

With the virtually infinite amount of information floating around on social media about current world affairs, it would be nice to be able to automatically parse the information and get the data you need related to which investments you should buy. We can solve this problem by building a program to automatically scrape the data and derive key details from it using AI.

We believe that we can get a better picture of what stocks will be if we can combine 2 things, well optimized stock data and a twitter api. The first part of this report will focus on the:

# Stock Data

Our stock data can be broadly broken down into 2 different categories:

- Daily Stock Data
- Historical Stock Data

This project initially started off by using Daily stock data entirely, but this did not provide enough variance to collect adequate stock data so we had to add historic data. We build out our data matrix by giving a bias of 1:1 daily:historical data, building out a matrix that takes both the volatile nature of todays markets and a matrix that has enough stock variance to make some sort of long term predictions

# Stock Data Collection Historical

Historical stock data can be easily gotten from [kaggle/stock_data](https://www.kaggle.com/datasets/jacksoncrow/stock-market-dataset?resource=download) and extracted from our sample number of stocks using the python code below:

```python
for ticker in tickers_df['tickers']:
        ticker = ticker.upper()
        filename = f"{ticker}.csv"

        if not os.path.isfile(filename):
            print(f"Warning: File {filename} does not exist. Skipping.")
            continue

        # Read the individual ticker CSV
        df = pd.read_csv(filename)

        # Select and rename the columns
        df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']].copy()
        df.rename(columns={
            'Date': 'timestamp',
            'Open': 'open',
            'High': 'high',
            'Low': 'low',
            'Close': 'close',
            'Volume': 'volume'
        }, inplace=True)

        # Add the ticker column
        df['ticker'] = ticker

        # Format timestamp to add '00:00:00'
        df['timestamp'] = df['timestamp'] + ' 00:00:00'

        # Reorder columns
        df = df[['ticker', 'timestamp', 'open', 'high', 'low', 'close', 'volume']]

        master_data.append(df)
```

This then can be fed into a variety of data cleaners, clippers, and modifiers to make the data interface with our combiner later.

# Stock Data Collection Daily

Large detailed stock datasets were hard to come by, so we had to come up with a strategy to build a large enough dataset within a few acceptance criteria:

- A large array of stocks, given in this project as "stock_tickers.csv" or S
- Stocks that were going to stay relatively stable, for this I selected a list of the companies with the largest market cap descending, and collected as large a data set as possible $(\max(size(S)))$
- Stocks that were freely available with the data from Alpha Vantage (the API used in this project) set of Alpha Vantage given $A$, condition $S \in A$
- Stocks will be picked from the top of the Nasdaq top stock list $S_i = N_i, i = {1,2,3,\dots,n}$
- Number of pulls the API will allow me to do from Wall Street close on Friday to open on Monday (don't know how to express this one as the API throttling seems to happen at random)

Provided mathematically as:

$$\max(size(S))$$

$$s.t.$$

$$S \in A$$

$$S_i = N_i, i = \{ 1,2,3,\dots,n \}$$

n = the number of entries I am able to pull during the weekend the NYSE is closed



## $$S_i = N_i, i = \{ 1,2,3,\dots,n \}$$
This was a very simple constraint to follow because it just required me to search the NASDAQ for the highest market cap companies and download the 6000 entry long csv of stock valuation and tickers, that I can isolate the tickers on ([nasdq.com/highest market cap](https://www.nasdaq.com/market-activity/index/spx/historical?page=1&rows_per_page=10&timeline=m1)). This data could not be directly used as the stock market is in a state of flux which necessitates a higher amount of granularity see constraint below.

## $$S \in A$$
This project relies upon the stock data that can be freely procured from the Alpha Vantage API ([alphavantage.co/documentation](https://www.alphavantage.co/documentation/)), this constraint wa probed by using the above constraint set, using error handling for checking if it wasn't in the set (after an 8 hour timeout, that was found to be enough for the API timeout to reset) :

```python
max_retries = 5

...

if not time_series:
            print(f"No time series data found for {symbol}.") #bad data catch
            return
```

## $$\max(size(S))$$
This function is more served as me running a few scripts periodically throughout the weekend to maximize the number of stocks that can be grabbed from the API. It is run wit this .bat file:

```bat
@echo off

echo Starting up the app...

python -m compileall -q .

python main.py

pause
```

But the pertinent code for this report and this class is this:
```python
for attempt in range(max_retries):
            for i, key in enumerate(keys):
                url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=30min&apikey={key}' ## url to alphavantage
                response = requests.get(url)
                data = response.json()

                if "Meta Data" in data and "Time Series (30min)" in data:
                    print(f"[Key {i + 1}] Success for {symbol}.")
                    break
                else:
                    print(f"[Key {i + 1}] failed or rate-limited. Trying next key...") ## catch if they are limiting my rates (although I am starting to think it is by static ip)

            else:
                print(f"[Attempt {attempt + 1}] All keys failed. Waiting {retry_delay / 60} minutes...")
                if attempt == 3: # ratchet catch to get all the data I could glean so far, will overwrite if I get more data
                    downloader = stockCSVDownloader()
                    downloader.move_to_downloads("stocks.csv") #actual downloader

                    printer = lastStockPrinter()
                    printer.move_last_stock_to_downloads(symbol, str(self.count))
                    continue
                elif attempt == 4:
                    time.sleep(retry_delay * 8) #long sleeper so I can afk this

                time.sleep(retry_delay)
                continue
            break
        else:
            print(f"Failed to fetch data for {symbol} after {max_retries} retries.")
            return
```

This peice of code queries the API and adds the stock data to the csv that will be staged and downloaded after 3 failed attempts (my "soft catch" that hopefully catches when the API starts to throttle me.) after it throttles me, it prints out the last stock to be queried, I turn on a vpn, and continue the process from the last stock ticker it printed out.

All stocks_# are then combined and fed into a variance calculator made in Julia, given below:

```python
using CSV
using DataFrames
using Statistics

df = CSV.read("top_stocks_data.csv", DataFrame)
grouped = groupby(df, :ticker)

results = DataFrame(
    ticker = String[],
    open_variance = Float64[],
    high_variance = Float64[],
    low_variance = Float64[],
    close_variance = Float64[],
    volume_variance = Float64[],
)

for g in grouped
    push!(results, (
        ticker = first(g.ticker),
        open_variance = var(g.open),
        high_variance = var(g.high),
        low_variance = var(g.low),
        close_variance = var(g.close),
        volume_variance = var(g.volume),
    ))
end

CSV.write("stock_variance.csv", results)

println("Variance data was written to stock_variance.csv.")
```

# Combining the stock data

Because these data sheets (.csv) are theoretically formatted correctly, I combined the via google sheets and uploaded them to then be optimized via our optimizer written in Julia to get the best stocks according to just stock calculation, non-considering the twitter sentiment that will effect the final verdict.

# Optimizing the data:
The code used to optimize the data can broadly broken down in a few steps
#### Uploading data
this basically uploads all the data from the .csv
```julia
# Load stock data
stock_data = CSV.read("master_stocks.csv", DataFrame)
```

#### Calculating returns
we need to calculate the approximate daily return using the formula: $$r_t = \frac{P_t - P_{t-1}}{P_{t-1}}$$
where:
- $r_t$ is the return at time t
- $P_t$ is the price at time t

```julia
function compute_returns(df)
    df = sort(df, :ticker)
    returns = [missing; diff(df.close) ./ df.close[1:end-1]]
    df[!, :return] = returns
    return df
end

returns_data = combine(groupby(stock_data, :ticker), compute_returns) # Compute returns
returns_wide = unstack(returns_data, :ticker, :return; combine=mean)
returns_wide_clean = dropmissing(returns_wide) # Drop rows with missing returns
```

#### Calculate covariance Matrix
When we have calculated the returns, we can compute the covariance matrix $\Sigma$ this is a square matrix that describes the covariances between different stocks in the portfolio. This matrix can be given by $$\sigma_{ij} = \text{Cov}(r_i, r_j) = \frac{1}{N-1} \sum_{t=1}^{N} (r_{i,t} - \bar{r}_i)(r_{j,t} - \bar{r}_j)$$
```julia
returns_matrix = Matrix{Float64}(returns_wide_clean[:, clean_stocks])
sigma = cov(returns_matrix)
```

#### Calculate Expected Returns $\mu$

The expected return (given in the vector $\mu$) is the difference between the open and close which yeilds:$$\mu_i = \frac{1}{T} \sum_{t=1}^{T} \frac{P_{i,\text{open},t} - P_{i,\text{close},t}}{P_{i,\text{open},t}}$$
```julia
mu = Float64[]

for stock in clean_stocks
    timepoints = subset(valid_stock_data, :ticker => ByRow(==(stock)))
    expected_return = 0.0
    for time in eachrow(timepoints)
        ret = (time[:open] - time[:close]) / time[:open]
        expected_return += ret
    end
    expected_return /= 100

    if ismissing(expected_return) || isnan(expected_return) || isinf(expected_return)
        println("stock dropped: $stock")
        continue
    end

    push!(mu, expected_return)
end
```

#### Optimization problem

Optimizer: Quadratic, where the objective is $\min(\text{risk})$ while getting a certian expected return, this function can be represented as: $$\text{Objective} = - \mu^T x + \lambda x^T \Sigma x$$
```julia
lambda = 0.5
model = Model(Ipopt.Optimizer)
n_stocks = length(mu)

@variable(model, x[1:n_stocks] >= 0)
@constraint(model, sum(x) == 1)
@objective(model, Min, -mu' * x + lambda * (x' * sigma * x))

optimize!(model)
```

#### Output to .txt/.csv
This simply uses julias inbuilt function to output.
```julia
stocks_info = [(clean_stocks[i], value(x[i]), mu[i]) for i in 1:n_stocks]
sorted_stocks = sort(stocks_info, by = x -> x[2], rev=true)

# Write to CSV
csv_filename = "topstocks.csv"
CSV.write(csv_filename, DataFrame(Stock = [stock[1] for stock in sorted_stocks],
                                 Allocation = [stock[2] for stock in sorted_stocks],
                                 ExpectedReturn = [stock[3] for stock in sorted_stocks]))

# Write to TXT
txt_filename = "topstocks.txt"
open(txt_filename, "w") do f
    println(f, "Top Stocks")
    for i in (length(sorted_stocks))
        stock = sorted_stocks[i]
        println(f, "$i. $(stock[1]) - Estimated Return: $(stock[3])")
    end
end
```